# The Tampa Bay Times and school performance

**Story:** [The story](http://www.tampabay.com/projects/2015/investigations/pinellas-failure-factories/), and [a critique](https://rogueedu.blogspot.com/2015/08/fcat-reading-scores-only-two-of-five.html)

**Author:** A million people, but I believe Nathaniel Lash did the data analysis

**Topics:** Linear Regression, Residuals

**Datasets**

* **0066897-gr04_rsch_2014.xls:** 4th grader pass rates for standardized tests, from Florida Dept of Education
* **FRL-1314-School-Web-Survey-3-Final.xls:** Free and reduced price lunch data, from Florida Dept of Education
* **MembershipSchoolRaceGender1415.xls:** School population by gender, from Florida Dept of Education

# What's the story?

We're trying to see what kind of effect race and poverty have on school test score data. Their actual story doesn't include a regression, but they used one behind the scenes for research.

## Imports

You'll want pandas and seaborn. You'll want want to set pandas to display a lot of columns and rows at a time.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [2]:
pd.options.display.max_columns = 200
# pd.reset_option('display.max_columns')

# Reading in our data

Read in the dataset, being sure to read in the district and school number as strings in case we need to merge on anything later. If pandas gets its way, it would read them in as integers and turn `0001` into `1`. This dataset includes school information, as well as

* The percentage of students who passed their 4th grade standardized reading test
* The percentage of students receiving free or reduced price lunch, as a proxy for poverty
* The percentage of students that are Black/African-American

In [3]:
testing_df = pd.read_excel('sources/0066897-gr04_rsch_2014.xls', skiprows=10, \
                              usecols=['District Number','District Name','School Number','School Name','Number of Students','Percentage Passing (Achievement Levels 3 and Above)'], \
                              dtype={'District Number':str,'School Number':str})
testing_df = testing_df.rename({'Percentage Passing (Achievement Levels 3 and Above)':'Testing Pct Passing','Number of Students':'Testing Total'}, axis=1)
testing_df = testing_df.drop(testing_df.index[0:2])
testing_df = testing_df[testing_df['Testing Pct Passing'] != '*'].reset_index(drop=True)
testing_df[['Testing Total', 'Testing Pct Passing']] = testing_df[['Testing Total', 'Testing Pct Passing']].apply(pd.to_numeric) 
testing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2079 entries, 0 to 2078
Data columns (total 6 columns):
District Number        2079 non-null object
District Name          2079 non-null object
School Number          2079 non-null object
School Name            2079 non-null object
Testing Total          2079 non-null int64
Testing Pct Passing    2079 non-null int64
dtypes: int64(2), object(4)
memory usage: 65.0+ KB


In [30]:
lunches_df = pd.read_excel('sources/FRL-1314-School-Web-Survey-3-Final.xls', 'School', skiprows=4, dtype='str')
lunches_df = lunches_df.drop(lunches_df.index[0])
lunches_df = lunches_df[lunches_df['Total Members'] != '*'].reset_index(drop=True)
lunches_df = lunches_df.rename({'School #':'School Number','District':'District Name','Total Members':'Lunch Total','Eligible for Free or Reduced Price Lunch':'Lunch Free','Unnamed: 6':'Lunch Reduced','Unnamed: 7':'Lunch Prov2','Unnamed: 8':'Lunch DirCert'}, axis=1)
lunches_df[['Lunch Total','Lunch Free','Lunch Reduced','Lunch Prov2','Lunch DirCert']] = lunches_df[['Lunch Total','Lunch Free','Lunch Reduced','Lunch Prov2','Lunch DirCert']].apply(pd.to_numeric)
lunches_df['Lunch Pct Eligible'] = (((lunches_df['Lunch Free'] + lunches_df['Lunch Reduced'] + lunches_df['Lunch Prov2'] + lunches_df['Lunch DirCert']) / lunches_df['Lunch Total']) *100).round().astype(np.int64)
lunches_df['District Number'] = lunches_df['District Number'].str.zfill(2)
lunches_df['School Number'] = lunches_df['School Number'].str.zfill(4)
lunches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3903 entries, 0 to 3902
Data columns (total 10 columns):
District Number       3903 non-null object
District Name         3903 non-null object
School Number         3903 non-null object
School Name           3903 non-null object
Lunch Total           3903 non-null int64
Lunch Free            3903 non-null int64
Lunch Reduced         3903 non-null int64
Lunch Prov2           3903 non-null int64
Lunch DirCert         3903 non-null int64
Lunch Pct Eligible    3903 non-null int64
dtypes: int64(6), object(4)
memory usage: 244.0+ KB


In [20]:
demogr_df = pd.read_excel('sources/MembershipSchoolRaceGender1415.xls', 'SCHOOL', skiprows=4, dtype='str')
demogr_df = demogr_df[demogr_df['Grade'] == '4']
demogr_df = demogr_df[demogr_df['Total'] != '*'].reset_index(drop=True)
demogr_df = demogr_df.replace(r'^\s*$', '0', regex=True)
demogr_df = demogr_df.rename({'District #':'District Number','District':'District Name','School #':'School Number','School ':'School Name','Total':'Demog Total','White':'Demog White','Black or African American':'Demog Black','Hispanic/Latino':'Demog Hisp','Asian':'Demog Asian','Native Hawaiian or Other Pacific Islander':'Demog PI','American Indian or Alaska Native':'Demog NA','Two or More Races':'Demog Multi','Female Total':'Demog Female','Male Total':'Demog Male'}, axis=1)
demogr_df[['Demog Total','Demog White','Demog Black','Demog Hisp','Demog Asian','Demog PI','Demog NA','Demog Multi','Demog Female','Demog Male']] = demogr_df[['Demog Total','Demog White','Demog Black','Demog Hisp','Demog Asian','Demog PI','Demog NA','Demog Multi','Demog Female','Demog Male']].apply(pd.to_numeric)
demogr_df['Demog Pct White'] = ((demogr_df['Demog White'] / demogr_df['Demog Total']) *100).round().fillna(0).astype(np.int64)
demogr_df['Demog Pct Black'] = ((demogr_df['Demog Black'] / demogr_df['Demog Total']) *100).round().fillna(0).astype(np.int64)
demogr_df['Demog Pct Hisp'] = ((demogr_df['Demog Hisp'] / demogr_df['Demog Total']) *100).round().fillna(0).astype(np.int64)
demogr_df['Demog Pct Minority'] = (((demogr_df['Demog Total'] - demogr_df['Demog White']) / demogr_df['Demog Total']) *100).round().fillna(0).astype(np.int64)
demogr_df = demogr_df.drop('Grade', axis=1)
demogr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3847 entries, 0 to 3846
Data columns (total 18 columns):
District Number       3847 non-null object
District Name         3847 non-null object
School Number         3847 non-null object
School Name           3847 non-null object
Demog White           3847 non-null int64
Demog Black           3847 non-null int64
Demog Hisp            3847 non-null int64
Demog Asian           3847 non-null int64
Demog PI              3847 non-null int64
Demog NA              3847 non-null int64
Demog Multi           3847 non-null int64
Demog Female          3847 non-null int64
Demog Male            3847 non-null int64
Demog Total           3847 non-null int64
Demog Pct White       3847 non-null int64
Demog Pct Black       3847 non-null int64
Demog Pct Hisp        3847 non-null int64
Demog Pct Minority    3847 non-null int64
dtypes: int64(14), object(4)
memory usage: 480.9+ KB


In [31]:
merge_df = testing_df
merge_df = merge_df.merge(lunches_df, how='left', left_on=['District Number','School Number'], right_on=['District Number','School Number'])
merge_df = merge_df.merge(demogr_df, how='inner', left_on=['District Number','School Number'], right_on=['District Number','School Number'])
merge_df = merge_df[['District Number','District Name_x','School Number','School Name_x','Testing Pct Passing','Lunch Pct Eligible','Demog Pct Black']]
merge_df = merge_df.rename({'District Name_x':'District Name','School Name_x':'School Name'}, axis=1)
# merge_df = merge_df.dropna(axis=0, how='any').reset_index(drop=True)
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2061 entries, 0 to 2060
Data columns (total 7 columns):
District Number        2061 non-null object
District Name          2061 non-null object
School Number          2061 non-null object
School Name            2061 non-null object
Testing Pct Passing    2061 non-null int64
Lunch Pct Eligible     2061 non-null int64
Demog Pct Black        2061 non-null int64
dtypes: int64(3), object(4)
memory usage: 96.6+ KB


In [49]:
merge_df.sample(10)

,District Number,District Name,School Number,School Name,Testing Pct Passing,Lunch Pct Eligible,Demog Pct Black
1152,36,LEE,4231,CAPE CORAL PREPARATORY & FITNESS AC,58,76,11
93,05,BREVARD,3071,SABAL ELEMENTARY SCHOOL,54,67,7
701,16,DUVAL,0911,SALLYE B. MATHIS ELEMENTARY SCHOOL,28,76,93
813,17,ESCAMBIA,2104,JACKIE HARRIS PREPARATORY ACADEMY,11,93,95
1176,37,LEON,1161,CANOPY OAKS ELEMENTARY SCHOOL,67,46,23
641,13,MIAMI DADE,5421,SUNSET PARK ELEMENTARY SCHOOL,71,74,2
782,16,DUVAL,7023,DUVAL VIRTUAL INSTRUCTION ACADEMY,50,31,40
225,06,BROWARD,3041,CORAL PARK ELEMENTARY SCHOOL,64,48,21
1512,50,PALM BEACH,0291,NORTHBORO ELEMENTARY SCHOOL,48,76,46
27,01,ALACHUA,0956,EXPRESSIONS LEARNING ARTS ACADEMY,92,12,7


### Converting to percentages

It's really easy to get mixed up later if we don't have our percentage columns as actual percents. Multiply any percentages that go 0-1 by 100 to turn them into 0-100 instead.

* **Tip:** Make sure your numbers are 1-100 after you multiply!

# Graphing our data

Use seaborn's `regplot` to plot the relationship between free/reduced lunch and percent passing, and the same with percent black.

* **Tip:** You can use `scatter_kws={'alpha':0.3}` to see things a bit more nicely

# Linear regression

Now let's be a little more exact: run a linear regression that takes into account both percent black and percent free or reduced.

* **Tip:** Use `.dropna()` to remove missing data
* **Tip:** Remember to use `sm.add_constant`!

## Describe the relationship coefficient using "real" words

For example, "For every X change, we get Y change"

# Overperformers and underperformers

The point of the regression is to predict the percent passing, right? We can use `result.predict()` to get the predicted passing rate for each school. Try to run it below: 

Now, let's **save that value into a new column**, we can call it `predicted_passing`.

### Confirm that Charles W. Duval had a predicted passing rate of 32.

## Now let's find the difference between the predicted passing rate and the actual passing rate

If we're being stats-y, this is called **the residual**. Save it into a new column called.... `residual`.

* **Tip:** Think real hard about which direction you should be subtracting in.

### Find the 10 schools that did much worse than predicted

* PRINCETON HOUSE CHARTER should be the worst, with PEPIN ACADEMIES below that

### Find the top 10 schools that did better than predicted

* PARKWAY MIDDLE SCHOOL should be the best, and PATHWAYS should be second

# What problems might our analysis have?

We brought in two things we thought would do a good job covering socioeconomics and demographic patterns. What else might we be missing?

* **Tip:** Pay attention to the names of the schools